# Introducing pandas

In [85]:
import pandas as pd

## Reading and Importing Data

We are going to use the House Sales in King County, USA dataset from Kaggle: https://www.kaggle.com/datasets/harlfoxem/housesalesprediction?select=kc_house_data.csv

In [86]:
df = pd.read_csv('kc_house_data.csv')

## Inspecting the Data

In [87]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

### Creating subset of original DataFrame

In [88]:
cols = ['id', 'price', 'view']
df_ = df[cols].copy()
df_.head()

,id,price,view
0,7129300520,221900.0,0
1,6414100192,538000.0,0
2,5631500400,180000.0,0
3,2487200875,604000.0,0
4,1954400510,510000.0,0


Checking details of our dataframe:

In [89]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      21613 non-null  int64  
 1   price   21613 non-null  float64
 2   view    21613 non-null  int64  
dtypes: float64(1), int64(2)
memory usage: 506.7 KB


'view' can be considered an ordinal categorical variable, but it is represented by integers in out dataset, we can change that. We can even make categories for the 'price' by binning.

## pandas apply()

**Syntax:** <br>
DataFrame.apply(func, axis=0, args=*tuple*, **kwargs)<br>
*kwargs: additional keyword pass to func in addition to the array/series.*


Let's use this built-in pandas method to change the 'view' column:

In [97]:

df_.view.value_counts().sort_index()

view
Average        963
Bad          19821
Excellent      319
Good           510
Name: count, dtype: int64

The 'view' variable takes 4 values, we can categorize them:

In [91]:
def change_view(n):
    if n == 0 or n == 1:
        return "Bad"
    elif n == 2:
        return "Average"
    elif n == 3:
        return "Good"
    else:
        return "Excellent"

Let's apply this function to the view column:

In [92]:
df_['view'] = df_['view'].apply(change_view)
df_.view.value_counts().sort_index()

view
Average        963
Bad          19821
Excellent      319
Good           510
Name: count, dtype: int64

In [93]:
df_.head()

,id,price,view
0,7129300520,221900.0,Bad
1,6414100192,538000.0,Bad
2,5631500400,180000.0,Bad
3,2487200875,604000.0,Bad
4,1954400510,510000.0,Bad


As observed, the view column now has string values. We can check again using info()

In [94]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      21613 non-null  int64  
 1   price   21613 non-null  float64
 2   view    21613 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 506.7+ KB


We will do something similar with the 'price' column. Let's check the price variable first:

In [95]:
df_['price'].describe().round(2)

count      21613.00
mean      540182.16
std       367362.23
min        75000.00
25%       321950.00
50%       450000.00
75%       645000.00
max      7700000.00
Name: price, dtype: float64

Now we create a function that returns a label for each price range and apply it to our column:

In [102]:
intervals = [75000, 300000, 600000, 900000, 1200000, 1500000, 8000000]
labels = ['< 300,000', '300,000 - 600,000', '600,000 - 900,000', '900,000 - 1,200,000', '1,200,000 - 1,500,000', '> 1,500,000']
# The following function iterates over the indices of intervals up to the second-to-last index and returns the corresponding label,
# and returns the last label for the last index in the interval.
def bin_price(price):
    for i in range(len(intervals)-1):
        if price <= intervals[i+1]:
            return labels[i]
    return labels[-1]
df_['price_bins'] = df_['price'].apply(bin_price)
df_.sort_values('price')

,id,price,view,price_bins
1149,3421079032,75000.0,Bad,"< 300,000"
15293,40000362,78000.0,Bad,"< 300,000"
465,8658300340,80000.0,Bad,"< 300,000"
16198,3028200080,81000.0,Bad,"< 300,000"
8274,3883800011,82000.0,Bad,"< 300,000"
...,...,...,...,...
1448,8907500070,5350000.0,Excellent,"> 1,500,000"
4411,2470100110,5570000.0,Bad,"> 1,500,000"
9254,9208900037,6890000.0,Excellent,"> 1,500,000"
3914,9808700762,7060000.0,Average,"> 1,500,000"


We used the apply() method to bin the price and added a column to our dataframe showing the price range for each row.